In [1]:
import pyodbc
import pandas as pd
import os
import datetime
from datetime import timedelta

In [2]:
os.getcwd()

'/home/aa/cloudJ/JB/PRD/scrapper/Scripts/Usa/Zaful'

In [3]:
path = 'Salida'
outputs = os.listdir(path)

In [4]:
fecha = datetime.date.today() - timedelta(days=0)

In [5]:
fecha

datetime.date(2020, 3, 12)

In [6]:
outputs = [path+"/"+xlsx for xlsx in outputs if xlsx.find(str(fecha)) != -1]

In [7]:
outputs = list(map(pd.read_excel,outputs))

In [8]:
df = pd.DataFrame([])
for chunk in outputs:
    df = df.append(chunk,sort=True)

In [9]:
len(df)#25263 23992

22682

In [10]:
df = df.dropna()
df = df.drop_duplicates()

In [11]:
df = df[df['precio_dto'] != -1]

In [12]:
df.sort_values('pagina_scraper',inplace=True)

In [13]:
df.reset_index(drop=True,inplace=True)

In [14]:
df['precio_dto'] = df['precio_dto'].apply(lambda x: x.split('USD')[1].strip()).astype(float)

In [15]:
df['precio_original'] = df['precio_original'].apply(lambda x: x.split('$')[1].strip()).astype(float)

In [16]:
df.to_excel(f"{path}/zaful_full-{fecha}.xlsx")

In [17]:
df.columns

Index(['color', 'descripcion', 'id_producto', 'img', 'pagina_scraper', 'pos',
       'precio_dto', 'precio_original', 'sexo', 'sub_categoria', 'talle',
       'tipo', 'url'],
      dtype='object')

In [18]:
df['origen'] = 'ZAFUL'
df['marca'] = 'ZAFUL'
df['fecha_alta'] = fecha
df['descripcion_aux'] = '0'
df['moneda'] = 'DOLAR US$'
df['stock'] = 0

In [19]:
#INSERT
server = 'tcp:192.168.1.6'
database ="planeamiento"
username = 'sa'
password = 'sa'
cnxn = pyodbc.connect('DRIVER={/opt/microsoft/msodbcsql17/lib64/libmsodbcsql-17.2.so.0.1};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password) 

In [20]:
maxCorrida = "select max(id_sc2_corrida)+1 from sc2_detalle WHERE origen ='ZAFUL' "
maxCorrida = pd.read_sql(maxCorrida,cnxn)

In [21]:
maxCorrida = int(maxCorrida[""][0])

In [22]:
maxCorrida

14

In [23]:
cursor = cnxn.cursor()
for index,row in df.iterrows():
    try:
        cursor.execute("INSERT INTO sc2_detalle([id_sc2_producto], [id_sc2_corrida],[talle],[marca],[tipo],[color],[sexo],[descripcion_aux],[descripcion],[stock],[moneda],[precio],[precio_original],[fecha_alta],[url_imagen],[url_producto],[origen],[tipo_vigente],[posicion],[pagina])values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                       row['id_producto'], 
                       maxCorrida, 
                       row['talle'],
                       row['marca'],
                       row['sub_categoria'],
                       row['color'],
                       row['sexo'],
                       row['descripcion_aux'],
                       row['descripcion'], 
                       row['stock'],
                       row['moneda'],
                       row['precio_dto'], 
                       row['precio_original'], 
                       row['fecha_alta'],
                       row['img'], 
                       row['url'],
                       row['origen'],
                       row['sub_categoria'],
                       row['pos'],
                       row['pagina_scraper']) 
        cnxn.commit()
    except:
        print('_'*100)
        print(row)
#cursor.close()
#cnxn.close()

In [24]:
index #23991

22681